### Statistic Tuning

In [ ]:
import os
import pandas as pd

In [ ]:
tuning_result_file = r"G:\AI\MonPot\debug\break_strategy_tuning_config.csv"

In [ ]:
df = pd.read_csv(tuning_result_file, index_col=0)
df.head()

In [ ]:
nlargest = df.nlargest(5, "TOTAL_PnL(%)")
nlargest

In [ ]:
for i, row in nlargest.iterrows():
    print(row["params"].replace("'", '"'), row["TOTAL_PnL(%)"])

### PnL Cumulative Visualization

In [ ]:
import os
import pandas as pd
from pandas import Timestamp
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [ ]:
def visualize_PnL(order_paths, names, colors):
    fig = make_subplots()
    fig.update_layout(
        plot_bgcolor="#2E4053",
        paper_bgcolor="#797D7F",
        font=dict(color="#F7F9F9"),
        xaxis=dict(showgrid=False),
        yaxis=dict(showgrid=False),
    )
    for order_path, name, color in zip(order_paths, names, colors):
        df_od = pd.read_csv(order_path)
        cum_PnL = []
        PnL = 0
        for i, row in df_od.iterrows():
            if row["status"] == "HIT_SL":
                pnl = round(abs(row["sl"] - row["entry"]) / row["entry"], 4)
                if (row["side"] == "SELL" and row["sl"] <= row["entry"]) or (
                    row["side"] == "BUY" and row["sl"] >= row["entry"]
                ):
                    PnL += pnl * 100
                else:
                    PnL -= pnl * 100
            elif row["status"] == "HIT_TP":
                PnL += row["reward_ratio"] * 100
            elif row["status"] == "STOPPED":
                PnL += eval(row["attrs"])["PnL"] * 100
            cum_PnL.append(PnL)

        fig.add_trace(
            go.Scatter(x=list(range(len(cum_PnL))), y=cum_PnL, mode="lines", line=dict(color=color, width=2), name=name)
        )
    return fig


In [ ]:
visualize_PnL(
    [r"debug\XAUUSDm\XAUUSDm_orders.csv"],
    ["XAUUSDm"],
    ["green"],
)
